# Tutorial to read a Galacticus HDF5 file.

In [1]:
import numpy as np

# Load GalacticusHDF5 class for reading/writing of Galacticus HDF5 files.
from galacticus.io import GalacticusHDF5

In [2]:
# Load Galacticus HDF5 file
ifile = "/Users/amerson/Data/Galacticus_Out/v0.9.4/wfirstSED_singleTree_lightcone/galacticus.hdf5"
GH5 = GalacticusHDF5(ifile,'r')

In [4]:
# The Galacticus HDF5 class has an attribute "outputs" that stores the information for the snapshots in this file.
for i in range(len(GH5.outputs.z)):
    print(str(GH5.outputs.iout[i])+" "+GH5.outputs.name[i]+" z = "+str(GH5.outputs.z[i]))

1 Output1 z = 2.0
2 Output2 z = 1.8
3 Output3 z = 1.6
4 Output4 z = 1.4
5 Output5 z = 1.2
6 Output6 z = 1.0


In [8]:
# These outputs are sub-directories in the HDF5 file where the galaxy data is stored. In each output the galaxy 
# data is stored in a sub-directory 'nodeData'.
print GH5.lsGroups("/")
print GH5.lsGroups("/Outputs")
print GH5.lsGroups("/Outputs/Output1")

['Build', 'Filters', 'Outputs', 'Parameters', 'Version', 'globalHistory', 'metaData']
['Output1', 'Output2', 'Output3', 'Output4', 'Output5', 'Output6']
['nodeData']


In [14]:
# For any redshift we can identify the snapshot output that is closest in redshift. For example, for a redshift 
# z = 1.3 we can find the redshift of the nearest snapshot to be:
z = 1.3
zsnap = GH5.nearestRedshift(z)
outputName = GH5.nearestOutputName(z)
print("Snapshot with nearest redshift is "+outputName+" with z = "+str(zsnap))

Snapshot with nearest redshift is Output5 with z = 1.2


In [13]:
# We can then select this snapshot output.
output = GH5.selectOutput(z)
print output.name

/Outputs/Output5


In [16]:
# We can check to see how many galaxies were output in this snapshot.
ngals = GH5.countGalaxies(z)
print("Number of galaxies = "+str(ngals))

Number of galaxies = 307


In [19]:
# Or, if we do not specifiy a redshift, we can count and sum the galaxies in all of the snapshots.
ngals = GH5.countGalaxies()
print("Total number of galaxies = "+str(ngals))

Total number of galaxies = 1061


In [21]:
# We can view the galaxy datasets in an output in three ways. Firstly we can do (printing only the first 10 properties):
datasets = output["nodeData"].keys()
print datasets[:10]

[u'angularWeight', u'basicMass', u'basicTimeLastIsolated', u'blackHoleCount', u'blackHoleMass', u'blackHoleSpin', u'darkMatterProfileScale', u'diskAbundancesGasMetals', u'diskAbundancesStellarMetals', u'diskAngularMomentum']


In [47]:
# Secondly we can do:
datasets = GH5.lsDatasets(output.name+"/nodeData")
print datasets[:10]

['angularWeight', 'basicMass', 'basicTimeLastIsolated', 'blackHoleCount', 'blackHoleMass', 'blackHoleSpin', 'darkMatterProfileScale', 'diskAbundancesGasMetals', 'diskAbundancesStellarMetals', 'diskAngularMomentum']


In [48]:
# Or thirdly we can do:
datasets = GH5.availableDatasets(z)
print datasets[:10]

['angularWeight', 'basicMass', 'basicTimeLastIsolated', 'blackHoleCount', 'blackHoleMass', 'blackHoleSpin', 'darkMatterProfileScale', 'diskAbundancesGasMetals', 'diskAbundancesStellarMetals', 'diskAngularMomentum']


In [50]:
# Reading galaxy proprerties can be done by directly reading from the HDF5 output, for example:
arr = np.array(output["nodeData/basicMass"])
print arr[:10]

[  5.18752000e+10   5.18752000e+10   2.93838049e+10   2.93838049e+10
   4.95172370e+10   4.95172370e+10   3.30114890e+10   3.30114890e+10
   3.30239637e+11   3.30239637e+11]


In [56]:
# Or by using the 'readGalaxies' function, where the 'props' keyword is a list of the properties that one
# wishes to read.
arr = GH5.readGalaxies(z,props=['basicMass'])
print type(arr)
print arr["basicMass"][:10]

<type 'numpy.ndarray'>
[  5.18752000e+10   5.18752000e+10   2.93838049e+10   2.93838049e+10
   4.95172370e+10   4.95172370e+10   3.30114890e+10   3.30114890e+10
   3.30239637e+11   3.30239637e+11]


In [58]:
# Using the 'readGalaxies' function, multiple properties can be read at once. Wildcards can be used to select
# multiple properties with similar names. The data is extracted into a numpy structured array.
arr = GH5.readGalaxies(z,props=['basicMass','blackHoleCount','*MassStellar'])
print type(arr)
print arr.dtype.names
print arr['diskMassStellar'][:10]

<type 'numpy.ndarray'>
('basicMass', 'blackHoleCount', 'diskMassStellar', 'spheroidMassStellar')
[  4.52305319e+08   4.52305319e+08   4.72490123e+07   4.72490123e+07
   2.21489325e+09   2.21489325e+09   2.04945869e+05   2.04945869e+05
   7.57263788e+09   7.57263788e+09]


In [63]:
# The Galacticus file is then closed using:
GH5.close()